<a href="https://colab.research.google.com/github/pinilDissanayaka/Facial-Expression-Recognition/blob/main/Notebook/Facial_Expression_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2 as cv
from glob import glob

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import plot_model

import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
import tensorflow
print("Tensorflow version:", tensorflow.__version__)

Tensorflow version: 2.14.0


In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download mahmoudima/mma-facial-expression

 97% 160M/166M [00:01<00:00, 190MB/s]
100% 166M/166M [00:01<00:00, 165MB/s]


In [ ]:
! unzip /content/mma-facial-expression.zip

In [7]:
TRAIN_DIR = '/content/MMAFEDB/train'
TEST_DIR = '/content/MMAFEDB/test'
VALID_DIR = '/content/MMAFEDB/valid'

In [9]:
train_labels = (os.listdir(TRAIN_DIR))
test_labels = (os.listdir(TEST_DIR))
valid_labels = (os.listdir(VALID_DIR))

In [10]:
print(f"Train labels = {train_labels}")
print(f"Test labels = {test_labels}")
print(f"VAlidation labels = {valid_labels}")

Train labels = ['fear', 'surprise', 'sad', 'happy', 'neutral', 'disgust', 'angry']
Test labels = ['fear', 'surprise', 'sad', 'happy', 'neutral', 'disgust', 'angry']
VAlidation labels = ['fear', 'surprise', 'sad', 'happy', 'neutral', 'disgust', 'angry']


In [15]:
train_labels == test_labels == valid_labels

True

In [23]:
train_img_paths = glob(TRAIN_DIR + '/*/*')
test_img_paths = glob(TEST_DIR + '/*/*')
valid_img_paths = glob(VALID_DIR + '/*/*')

In [24]:
train_imgs = len(train_img_paths)
test_imgs = len(test_img_paths)
valid_imgs = len(valid_img_paths)

In [25]:
print(f"For train purpose we have {train_imgs} images")
print(f"For test purpose we have {test_imgs} images")
print(f"For valid purpose we have {valid_imgs} images")

For train purpose we have 92968 images
For test purpose we have 17356 images
For valid purpose we have 17356 images


In [35]:
train_img_shapes = set()
test_img_shapes = set()
valid_img_shapes = set()

for i in train_img_paths:
  img = cv.imread(i)
  img_shape = img.shape
  train_img_shapes.add(img_shape)


for i in test_img_paths:
  img = cv.imread(i)
  img_shape = img.shape
  test_img_shapes.add(img_shape)


for i in valid_img_paths:
  img = cv.imread(i)
  img_shape = img.shape
  valid_img_shapes.add(img_shape)

In [36]:
print(f"Train image size : {train_img_shapes}")
print(f"Test image size : {test_img_shapes}")
print(f"Validation image size : {valid_img_shapes}")

Train image size : {(48, 48, 3)}
Test image size : {(48, 48, 3)}
Validation image size : {(48, 48, 3)}


In [37]:
IMG_SIZE = [48, 48, 3]